# update_df_varmap

维护`homo/df_varmap.csv`,该文件中存储源于biomart的物种间同源信息的文件路径

最后更新2024年4月2日

# [biomart](https://www.ensembl.org/biomart/martview)

In [1]:
import sys
from pathlib import Path
p_root = Path('.').absolute().parent
None if str(p_root) in sys.path else sys.path.append(str(p_root))

In [2]:
from func import group_agg,pd,np

/public/workspace/licanchengup/apps/miniconda3/envs/publish/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-02 12:43:44.701507: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



-------------------------help-------------------------
> parameter
    p_root	[name] res_publish
        p_run, p_plot, p_res, p_cache
    p_df_varmap
    map_sp_reverse
    rng
> run
    run_cross_species_models
    h5ad_to_mtx
    load_adata
    get_path_varmap
    find_path_from_para
    load_normalized_adata

> res
    get_test_result_df
    get_res_obs
    show_umap

> plot
    get_color_map
    show_color_map
    show_color
    plot_umap
    savefig



In [3]:
df = pd.DataFrame(
    {'path': p_root.joinpath('homo').iterdir()}
)
df['path'] = df['path'].apply(lambda x:x.relative_to(p_root.joinpath('homo')))
df = df[df['path'].apply(lambda x: x.match('*_to_*'))]
df['name'] = df['path'].apply(lambda x: x.stem)

df = df.join(df['name'].str.extract(
    '^(?P<sp_ref>[^_]+)_to_(?P<sp_que>[^_]+)$'))
df['tag'] = df['name'].str.split('_to_').apply(np.sort).str.join('_to_')
df['is_reverse'] = df.eval('tag != name')
display(group_agg(df, 'tag,is_reverse'.split(','), {'is_reverse': ['count']}).pivot(
    index='tag', columns='is_reverse', values='is_reverse_count'
).style.highlight_null(color='red'))
display(df.head(2), df.shape)

is_reverse,False,True
tag,,
chicken_to_zebrafish,1,1
human_to_macaque,1,1
human_to_mouse,1,1
human_to_zebrafish,1,1
macaque_to_mouse,1,1
macaque_to_pig,1,1
mouse_to_pig,1,1
mouse_to_zebrafish,1,1
xenopus_to_zebrafish,1,1


,path,name,sp_ref,sp_que,tag,is_reverse
0,zebrafish_to_mouse.txt,zebrafish_to_mouse,zebrafish,mouse,mouse_to_zebrafish,True
1,zebrafish_to_xenopus.txt,zebrafish_to_xenopus,zebrafish,xenopus,xenopus_to_zebrafish,True


(18, 6)

In [4]:
p_root

PosixPath('/public/workspace/licanchengup/download/res_publish')

In [5]:
df.loc[:, 'path,name,sp_ref,sp_que'.split(',')].to_csv(
    p_root.joinpath('homo','df_varmap.csv'), index=False)
print('\n[finish]\n'.center(100,'-'))

---------------------------------------------
[finish]
---------------------------------------------
